# Integração: Desmatamento, PIB, Queimadas, Bolsa Família, Plantações e Rebanho Bovino no Pará

Este notebook integra e trata dados socioeconômicos e ambientais do estado do Pará, organizando-os para análises futuras.

## 1. Importação de Bibliotecas

Importa as bibliotecas necessárias para manipulação de dados e modelagem.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

## 2. Carregamento e Integração dos Dados de Desmatamento e PIB

Carrega os dados de desmatamento e PIB, realize o merge e ajuste os nomes das colunas para facilitar o uso.

In [2]:
# Carrega os dados de desmatamento e PIB
desmatamento = pd.read_csv('../data/DESMATAMENTO/df_desmatamento_ajustado.csv', sep=';', encoding='utf-8')
pib_2002_2021_pa = pd.read_csv('../data/PIB/PIB_2002_2021_PA.csv', encoding='utf-8')

# Realiza o merge dos dados pelo ano e código IBGE
desmatamento_pib = pd.merge(
    left=desmatamento,
    right=pib_2002_2021_pa,
    how='left',
    left_on=['year', 'geocode_ibge'],
    right_on=['Ano', 'Código do Município']
).drop(columns=['Ano', 'Código do Município', 'Nome do Município'])

In [3]:
# Visualiza as colunas para conferência
desmatamento_pib.columns

Index(['year', 'areakm', 'municipality', 'geocode_ibge', 'state',
       'FullLocation',
       'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)',
       'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)'],
      dtype='object')

In [4]:
# Remove coluna de estado e renomeia colunas para padronização
desmatamento_pib = desmatamento_pib.drop(columns=['state'])

desmatamento_pib = desmatamento_pib.rename(columns={
    'year': 'Ano',
    'areakm': 'Desmatamento (km²)',
    'geocode_ibge': 'Código IBGE',
    'municipality': 'Município',
    'FullLocation': 'Localização',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB per capita (R$)',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB (R$ 1.000)',
    'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)' : 'VAB Agropecuária (R$ 1.000)',
    'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)' : 'VAB Indústria (R$ 1.000)',
    'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)' : 'VAB Serviços (R$ 1.000)',
})

# Seleciona apenas as colunas relevantes
desmatamento_pib = desmatamento_pib[['Município', 'Ano', 'Código IBGE', 'Desmatamento (km²)', 'PIB per capita (R$)', 'PIB (R$ 1.000)', 'VAB Agropecuária (R$ 1.000)', 'VAB Indústria (R$ 1.000)', 'VAB Serviços (R$ 1.000)']]

## 3. Preenchimento de Valores Ausentes via Extrapolação Linear

Utilize regressão linear para preencher valores ausentes de PIB e VAB por município.

In [5]:
# Função para extrapolar valores ausentes de uma coluna numérica por município usando regressão linear
def extrapolar_pib(df, coluna='PIB per capita (R$)'):
    df = df.copy()
    for municipio in df['Município'].unique():
        dados = df[df['Município'] == municipio]
        mask = dados[coluna].notna()
        if mask.sum() >= 2:  # precisa de pelo menos 2 pontos para regressão
            X = dados.loc[mask, 'Ano'].values.reshape(-1, 1)
            y = dados.loc[mask, coluna].values
            model = LinearRegression()
            model.fit(X, y)
            anos_faltantes = dados.loc[~mask, 'Ano']
            if not anos_faltantes.empty:
                pred = model.predict(anos_faltantes.values.reshape(-1, 1))
                df.loc[(df['Município'] == municipio) & (df['Ano'].isin(anos_faltantes)), coluna] = pred
    return df

# Extrapola e arredonda os valores de PIB per capita e PIB total
desmatamento_pib = extrapolar_pib(desmatamento_pib, 'PIB per capita (R$)')
desmatamento_pib['PIB per capita (R$)'] = desmatamento_pib['PIB per capita (R$)'].round(2)

desmatamento_pib = extrapolar_pib(desmatamento_pib, 'PIB (R$ 1.000)')
desmatamento_pib['PIB (R$ 1.000)'] = desmatamento_pib['PIB (R$ 1.000)'].round(2)

# Extrapola e arredonda os valores de VAB para cada setor
for coluna in ['VAB Agropecuária (R$ 1.000)', 'VAB Indústria (R$ 1.000)', 'VAB Serviços (R$ 1.000)']:
    desmatamento_pib = extrapolar_pib(desmatamento_pib, coluna)
    desmatamento_pib[coluna] = desmatamento_pib[coluna].round(2)

In [6]:
# Calcula a população estimada a partir do PIB total e PIB per capita
desmatamento_pib['População'] = (desmatamento_pib['PIB (R$ 1.000)'] / desmatamento_pib['PIB per capita (R$)']) * 1000
desmatamento_pib['População'] = desmatamento_pib['População'].round(0).astype(int)

## 4. Integração dos Dados de Queimadas

Adiciona os dados de focos de queimadas por município e ano.

In [7]:
# Carrega dados de queimadas
queimadas = pd.read_csv('../data/QUEIMADAS/focos_br_pa_ref_2008_2024.csv', encoding='utf-8')

In [8]:
# Prepara coluna para merge (município em caixa alta)
desmatamento_pib['municipioUPPERCASE'] = desmatamento_pib['Município'].str.upper()

In [9]:
# Realiza merge com dados de queimadas
desmatamento_pib_queimadas = pd.merge(
    desmatamento_pib,
    queimadas,
    how='left',
    left_on=['Ano', 'municipioUPPERCASE'],
    right_on=['Ano', 'municipio']
)

In [10]:
# Ajusta colunas e tipos após merge
desmatamento_pib_queimadas.drop(columns=['municipioUPPERCASE', 'municipio'], inplace=True)
desmatamento_pib_queimadas.rename(columns={'focos': 'Focos de Queimada'}, inplace=True)
desmatamento_pib_queimadas['Focos de Queimada'] = desmatamento_pib_queimadas['Focos de Queimada'].fillna(0).astype(int)

## 5. Integração dos Dados do Bolsa Família

Adiciona o total de benefícios do Bolsa Família por município e ano.

In [11]:
# Carrega dados do Bolsa Família
bolsa_familia = pd.read_csv('../data/BOLSAFAMILIA/visdata3-download-11-05-2025 133427.csv', encoding='latin1')

In [12]:
# Seleciona e renomeia colunas relevantes
bolsa_familia = bolsa_familia[['Código', 'Unidade Territorial', 'Referência', 'Total de Benefícios Básicos']]
bolsa_familia = bolsa_familia.rename(columns={
    'Código': 'Código IBGE',
    'Unidade Territorial': 'Município',
    'Referência': 'Ano',
    'Total de Benefícios Básicos': 'Total de Benefícios Básicos (Bolsa Família)',
})

In [13]:
# Prepara coluna para merge (município em caixa alta)
desmatamento_pib_queimadas['MunicípioUPPERCASE'] = desmatamento_pib_queimadas['Município'].str.upper()

# Realiza merge com dados do Bolsa Família
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas,
    bolsa_familia,
    how='left',
    left_on=['Ano', 'MunicípioUPPERCASE'],
    right_on=['Ano', 'Município'],
    suffixes=('', '_bolsa')
)

In [14]:
# Remove colunas auxiliares e preenche valores ausentes via extrapolação
desmatamento_pib_queimadas_bolsa = desmatamento_pib_queimadas_bolsa.drop(columns=['MunicípioUPPERCASE', 'Código IBGE_bolsa', 'Município_bolsa'])

desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Total de Benefícios Básicos (Bolsa Família)')
desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'] = desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'].fillna(0).round(0).astype(int)

## 6. Integração dos Dados de Plantações

Adiciona a área plantada de soja e milho por município e ano.

In [15]:
# Carrega dados de plantações
soja_pa = pd.read_csv('../data/PLANTACOES/soja_pa.csv', encoding='utf-8')
milho_pa = pd.read_csv('../data/PLANTACOES/milho_pa.csv', encoding='utf-8')

In [16]:
# Adiciona área plantada de soja
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    soja_pa.rename(columns={'Área plantada (ha)': 'Área plantada soja (ha)'}),
    how='left',
    on=['Município', 'Ano']
)

# Adiciona área plantada de milho
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    milho_pa.rename(columns={'Área plantada (ha)': 'Área plantada milho (ha)'}),
    how='left',
    on=['Município', 'Ano']
)

# Preenche valores ausentes e ajusta negativos para zero
desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Área plantada soja (ha)')
desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'].fillna(0).round(0).astype(int)
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'] < 0, 'Área plantada soja (ha)'] = 0

desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Área plantada milho (ha)')
desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'].fillna(0).round(0).astype(int)
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'] < 0, 'Área plantada milho (ha)'] = 0

## 7. Integração dos Dados de Rebanho Bovino

Adicione o total do rebanho bovino por município e ano.

In [17]:
# Carrega dados de rebanho bovino
bovino_pa = pd.read_csv('../data/REBANHO/bovino_pa.csv', encoding='utf-8')

In [18]:
# Adiciona o total do rebanho bovino
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    bovino_pa,
    how='left',
    on=['Município', 'Ano']
)

# Preenche valores ausentes e ajusta negativos para zero
desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Total Rebanho (Bovino)')
desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'] = desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'].fillna(0).round(0).astype(int)
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'] < 0, 'Total Rebanho (Bovino)'] = 0

In [19]:
# Salva o arquivo final consolidado
desmatamento_pib_queimadas_bolsa.to_csv('../data/RESULTADOS/df_final.csv', sep=',', index=False, encoding='utf-8')

## 8. Visualização Final dos Dados

Visualize o DataFrame consolidado para conferência.

In [20]:
# Exibe o DataFrame final consolidado
desmatamento_pib_queimadas_bolsa

,Município,Ano,Código IBGE,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,Focos de Queimada,Total de Benefícios Básicos (Bolsa Família),Área plantada soja (ha),Área plantada milho (ha),Total Rebanho (Bovino)
0,Abaetetuba,2008,1500107,1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,194,12404,0,50,4194
1,Abaetetuba,2009,1500107,2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,190,14743,0,50,3438
2,Abaetetuba,2010,1500107,3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,162,16672,0,80,2263
3,Abaetetuba,2011,1500107,0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,130,18705,0,100,1527
4,Abaetetuba,2012,1500107,2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,153,20719,0,150,1859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,612,6374,0,100,149900
2444,Óbidos,2021,1505106,29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,377,6148,0,100,160052
2445,Óbidos,2022,1505106,31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,428,6681,0,100,172430
2446,Óbidos,2023,1505106,23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,1249,6810,0,50,180010
